In [3]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
import numba
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
from sklearn import svm
from scipy.linalg import norm
import numpy as np 
import os

In [ ]:
X, y = make_classification(n_samples=1000, n_features=5, n_informative=5, n_redundant=0, random_state=42)

# Train a Logistic Regression model (or any other classifier)
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [5]:
# Reading in the dataset
df=pd.read_csv('SVM_Dataset2.csv')

# svm_classifier.fit(df[['x1', 'x2']], df[['y']])

In [8]:
def broadcast_compute(Z, grid, epsilon): 
    for i in range(len(grid) - 1):
        # Compute broadcasted points (difference between grid[i,:] and all other points)
        broadcasted_pts = grid[i, :] - grid  # This creates a matrix of differences

        # Compute the row-wise norm of broadcasted points
        norm_broadcasted_pts = np.linalg.norm(broadcasted_pts, axis=1)

        # Compute the difference in predictions
        Z_vec = Z[i] - Z

        # Create a mask where the norm is less than epsilon and Z_vec equals -1
        mask = (norm_broadcasted_pts < epsilon) & Z_vec != 0

        # Find the indices where the mask is true
        masked_indices = np.where(mask)[0]  # Use [0] to extract the array of indices from the tuple

        # Compute new points as the midpoint between grid[i,:] and the masked points
        new_pts = (grid[i, :] + grid[masked_indices]) / 2

        # Append new points to the boundary_points array
        boundary_points = np.append(boundary_points, new_pts, axis=0)
    
    boundary_points = boundary_points[1:,:]
    return boundary_points

def compute_decision_boundary_points_gpu(model, X, resolution=100, epsilon=0.01):
    n_features = X.shape[1]    
    grid = np.zeros((resolution ** n_features, n_features))
    for i, feature in enumerate(range(n_features)):
        cp_array = np.linspace(X[:, feature].min() - 1, X[:, feature].max() + 1, resolution ** (n_features)).reshape(-1)
        grid[:, i] = cp_array
    Z = np.asarray(model.predict(grid))
    return broadcast_compute(Z, grid, epsilon)

In [27]:
from sklearn.neighbors import KDTree
import cupy as cp

def find_boundary_kdtree(grid, Z, epsilon):
    boundary_points = []
    grid = np.asarray(grid)
    Z = np.asarray(Z)

    # Step 1: Group points by label
    unique_labels = np.unique(Z)
    label_to_points = {label: grid[Z == label] for label in unique_labels}

    # Step 2: Check all unordered label pairs
    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            label_a, label_b = unique_labels[i], unique_labels[j]
            cluster_a = label_to_points[label_a]
            cluster_b = label_to_points[label_b]

            # Build KDTree on cluster_b
            tree_b = KDTree(cluster_b)

            for point in cluster_a:
                # Query for neighbors in cluster_b within epsilon
                idxs = tree_b.query_radius([point], r=epsilon)[0]
                for idx in idxs:
                    match_point = cluster_b[idx]
                    midpoint = (point + match_point) / 2
                    boundary_points.append(midpoint)

    return np.array(boundary_points)

@numba.njit
def cartesian_product_numba(arrays, out):
    n_arrays = len(arrays)
    shape = [len(a) for a in arrays]
    for idx in range(out.shape[0]):
        remainder = idx
        for dim in range(n_arrays - 1, -1, -1):
            size = shape[dim]
            out[idx, dim] = arrays[dim][remainder % size]
            remainder = remainder // size

@cuda.jit
def cartesian_product_cuda(input_2d, n_arrays, out):
    idx = cuda.grid(1)
    out_size = out.shape[0]
    
    if idx >= out_size:
        return

    stride = 1
    for i in range(n_arrays - 1, -1, -1):
        arr_len = input_2d.shape[1]
        pos = (idx // stride) % arr_len
        out[idx, i] = input_2d[i, pos]
        stride *= arr_len


def compute_cartesian_product_cuda(arrays):
    # Ensure uniform lengths (or pad if necessary)
    max_len = max(len(a) for a in arrays)
    padded = np.array([np.pad(a, (0, max_len - len(a)), constant_values=0) for a in arrays], dtype=np.float32)
    
    n_arrays = len(arrays)
    out_size = np.prod([len(a) for a in arrays])
    
    out = cuda.device_array((out_size, n_arrays), dtype=np.float32)
    
    d_input = cuda.to_device(padded)
    
    threads_per_block = 128
    blocks_per_grid = (out_size + threads_per_block - 1) // threads_per_block
    
    cartesian_product_cuda[blocks_per_grid, threads_per_block](d_input, n_arrays, out)
    
    result = out.copy_to_host()
    
    return result


@cuda.jit 
def find_boundary_gpu(grid, Z, epsilon, boundary_points, count):
    i = cuda.grid(1)
    n = grid.shape[0]

    if i < n:
        for j in range(i + 1, n):
            dist = 0.0
            for k in range(grid.shape[1]):
                diff = grid[i, k] - grid[j, k]
                dist += diff * diff
            dist = dist ** 0.5

            if dist < epsilon and Z[i] != Z[j]:
                idx = cuda.atomic.add(count, 0, 1)
                for d in range(grid.shape[1]):
                    boundary_points[idx, d] = (grid[i, d] + grid[j, d]) / 2


def compute_decision_boundary_points_cpu(model, X, resolution=100, epsilon=0.01):
    """
    Compute decision boundary points in the high-dimensional feature space.
    Args:
        model: Trained classifier.
        X: Input data (n_samples, n_features).
        resolution: Number of points to sample along each feature axis.
        epsilon: Small step size to detect class changes.
    Returns:
        boundary_points: Array of points near the decision boundary.
    """
    n_features = X.shape[1]
    linspaces = [] 
    
    for i in range(n_features):
        dim =  np.linspace(X.iloc[:, i].min(), X.iloc[:, i].max(), resolution)
        linspaces.append(dim) 

    total_size = np.prod([len(arr) for arr in linspaces], dtype=np.int64)

    out = np.empty((total_size, len(linspaces)), dtype=np.float64)
    arrays = tuple(np.asarray(arr) for arr in linspaces)
    cartesian_product_numba(arrays, out)
    
    Z = model.predict(out)
    boundary_points = find_boundary_kdtree(out, Z, epsilon)
    boundary_df = pd.DataFrame(boundary_points, columns=X.columns)
    return boundary_df

def find_boundary_kdtree_cuda(grid, Z, epsilon):
    boundary_points = []
    grid = cp.asarray(grid)  # Move grid to GPU
    Z = cp.asarray(Z)  # Move labels to GPU
    
    # Step 1: Group points by label
    unique_labels = cp.unique(Z)
    label_to_points = {label.item(): grid[Z == label] for label in unique_labels}
    
    # Step 2: Check all unordered label pairs
    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            label_a, label_b = unique_labels[i], unique_labels[j]
            cluster_a = label_to_points[label_a.item()]
            cluster_b = label_to_points[label_b.item()]

            # Step 3: Compute distances between points in cluster_a and cluster_b
            # Broadcasting for pairwise distance computation
            diff = cp.expand_dims(cluster_a, 1) - cp.expand_dims(cluster_b, 0)
            distances = cp.linalg.norm(diff, axis=-1)  # Compute Euclidean distance
            
            # Step 4: Get the points that are within the epsilon radius
            indices = cp.where(distances <= epsilon)
            
            # Step 5: Compute the midpoints of the boundary points
            for idx in zip(*indices):
                point_a = cluster_a[idx[0]]
                point_b = cluster_b[idx[1]]
                midpoint = (point_a + point_b) / 2
                boundary_points.append(midpoint.get())  # Move result back to CPU

    return np.array(boundary_points)

def compute_decision_boundary_points_gpu(model, X, resolution=100, epsilon=0.01):
    """
    Compute decision boundary points in the high-dimensional feature space.
    Args:
        model: Trained classifier.
        X: Input data (n_samples, n_features).
        resolution: Number of points to sample along each feature axis.
        epsilon: Small step size to detect class changes.
    Returns:
        boundary_points: Array of points near the decision boundary.
    """
    n_features = X.shape[1]
    linspaces = [] 
    
    for i in range(n_features):
        dim =  np.linspace(X.iloc[:, i].min(), X.iloc[:, i].max(), resolution)
        linspaces.append(dim) 

    total_size = np.prod([len(arr) for arr in linspaces], dtype=np.int64)

    out = np.empty((total_size, len(linspaces)), dtype=np.float64)
    arrays = tuple(np.asarray(arr) for arr in linspaces)
    cartesian_product_numba(arrays, out)
    
    Z = model.predict(out)
    boundary_points = find_boundary_kdtree_cuda(out, Z, epsilon)
    boundary_df = pd.DataFrame(boundary_points, columns=X.columns)
    return boundary_df

# Experiments of the Computation Costs for CPU only (SVM classifier)

$\textbf{ CPU used: AMD Ryzen 9 7950X 16-Core processor}$

- 50 Resolution (R = 50) with CPU only 
- 47 minutes - Does not finish 

- 25 Resolution (R = 25) with CPU only 
- 23 minutes - Does not finish 

- 20 Resolution (R = 20) with CPU only 
- 20 minutes - Does not finish

- 15 Resolution (R = 15) with CPU only 
- 26 minutes - Does not finish 

- 12 Resolution (R = 12) with CPU only 
- 6 minute runtime Done

- 10 Resolution (R = 10) with CPU only 
- 85 seconds Done

- 7 Resolution (R = 7) with CPU only
- 5 seconds Done

- 5 Resolution (R = 5) with CPU only 
- 0.3 second runtime

# Experiments with CPU only (with Numba) -- 4 features

- 20 Resolution (R = 20) with CPU 
- Over 10 minutes - NOT DONE

- 17 Resolution (R = 17) with CPU 
- 6 minute runtime

- 15 Resolution (R = 15) with CPU 
- 2 minute runtime

- 10 Resolution (R = 10) with CPU 
- 6 second runtime

In [ ]:
svm_classifier = svm.SVC(kernel='poly',C=10, degree=2, probability=True)
svm_classifier.fit(df[['x1', 'x2']], df['y'].values)

# Compute decision boundary points considering all features
boundary_points = compute_decision_boundary_points_cpu(svm_classifier, df[['x1','x2']], resolution=250, epsilon=0.1)
 
# Print the decision boundary points
print("Decision Boundary Points (All Features):")
print(boundary_points)
print(boundary_points.shape)

boundary_points = boundary_points.drop_duplicates(subset='x1')
bound_x, bound_y = boundary_points.iloc[:,0], boundary_points.iloc[:, 1]

N = 1000000
lower_boundx, upper_boundx = np.min(bound_x), np.max(bound_x)
f = interp1d(bound_x, bound_y, kind='cubic')
X_pred = np.linspace(lower_boundx, upper_boundx, N)[:, np.newaxis]
Y_pred = f(X_pred)
lower_boundy, upper_boundy = np.min(bound_y), np.max(bound_y)
x_min, x_max = bound_x.min(), bound_x.max()
y_min, y_max = bound_y.min(), bound_y.max()

X_pred = np.clip(X_pred, x_min, x_max)
Y_pred = np.clip(Y_pred, y_min, y_max)
plt.plot(X_pred, Y_pred, color='black')

# undesired_datapt = np.array([undesired_coords[0], undesired_coords[1]])
# optimal_datapt = closest_point(undesired_datapt, contour=contours)

(250,)
(250,)


c:\Users\Compuworld\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [9]:
print("Decision Boundary Points (Probability Across Both classes):")
svm_classifier.predict_proba(boundary_points) 

Decision Boundary Points (Probability Across Both classes):


array([[0.30758487, 0.69241513],
       [0.30778302, 0.69221698],
       [0.31262566, 0.68737434],
       ...,
       [0.31226139, 0.68773861],
       [0.31102366, 0.68897634],
       [0.30978129, 0.69021871]])

# Experiments with GPU (with Numba) 

- 20 Resolution (R = 20) with GPU 
- 5.9 second runtime

- 15 Resolution (R = 15) with GPU 
- 1.0 second runtime

In [14]:
svm_classifier = svm.SVC(kernel='poly',C=10, degree=2, probability=True)
svm_classifier.fit(df[['x1', 'x2']], df['y'].values)
# Compute decision boundary points considering all features
boundary_points = compute_decision_boundary_points_gpu(svm_classifier, df[['x1','x2']], resolution=500, epsilon=0.1)
 
# Print the decision boundary points
print("Decision Boundary Points (All Features):")
print(boundary_points)
print(boundary_points.shape)

N = 1000000
lower_boundx, upper_boundx = np.min(bound_x), np.max(bound_x)
f = interp1d(bound_x, bound_y, kind='cubic')
X_pred = np.linspace(lower_boundx, upper_boundx, N)[:, np.newaxis]
Y_pred = f(X_pred)
lower_boundy, upper_boundy = np.min(bound_y), np.max(bound_y)
x_min, x_max = bound_x.min(), bound_x.max()
y_min, y_max = bound_y.min(), bound_y.max()

X_pred = np.clip(X_pred, x_min, x_max)
Y_pred = np.clip(Y_pred, y_min, y_max)
plt.plot(X_pred, Y_pred, color='black')

[ 4.          4.0240481   4.04809619 ... 15.95190381 15.9759519
 16.        ]
[ 5.          5.03006012  5.06012024 ... 19.93987976 19.96993988
 20.        ]
(250000, 2)


c:\Users\Compuworld\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Decision Boundary Points (All Features):
[[ 4.577154  15.17535  ]
 [ 4.3366733 15.836674 ]
 [ 4.264529  16.047094 ]
 ...
 [15.987976  11.102204 ]
 [15.987976  11.087173 ]
 [15.987976  11.117235 ]]
(19452, 2)


NameError: name 'bound_x' is not defined

In [12]:
print("Decision Boundary Points (Probability Across Both classes):")
svm_classifier.predict_proba(boundary_points) 

Decision Boundary Points (Probability Across Both classes):


c:\Users\Compuworld\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([[0.31208524, 0.68791476],
       [0.311943  , 0.688057  ],
       [0.31250777, 0.68749223],
       ...,
       [0.31224754, 0.68775246],
       [0.31326464, 0.68673536],
       [0.31326464, 0.68673536]])

# CPU only for logistic regression 

4 features, 15 resolution - CPU 
- 2.5 minute runtime -> 1.2 seconds

5 features, 20 resolution - CPU
- Hour + -> 1 minute 55 second runtime O(n log n) solution

In [28]:
# Compute decision boundary points considering all features
boundary_points = compute_decision_boundary_points_cpu(model, pd.DataFrame(data=X, columns=["x"+str(i) for i in range(X.shape[1])]), resolution=25, epsilon=0.5)
 
# Print the decision boundary points
print("Decision Boundary Points (All Features):")
print(boundary_points)
print(boundary_points.shape)

KeyboardInterrupt: 

In [26]:
# Compute decision boundary points considering all features
boundary_points = compute_decision_boundary_points_gpu(model, pd.DataFrame(data=X, columns=["x"+str(i) for i in range(X.shape[1])]), resolution=20, epsilon=0.1)
 
# Print the decision boundary points
print("Decision Boundary Points (All Features):")
print(boundary_points)
print(boundary_points.shape)

OutOfMemoryError: Out of memory allocating 48,924,628,392,713,728 bytes (allocated so far: 10,096,605,696 bytes).

In [7]:
model.predict_proba(boundary_points)

array([[0.5111058 , 0.4888942 ],
       [0.49046014, 0.50953986],
       [0.48753307, 0.51246693],
       ...,
       [0.47453406, 0.52546594],
       [0.47745585, 0.52254415],
       [0.5068743 , 0.4931257 ]])

In [11]:
from scipy.interpolate import griddata

n_features = X.shape[1]
print(n_features)
ranges = [np.linspace(X[:, j].min() - 1, X[:, j].max() + 1, 100) for j in range(n_features)]
grids = np.meshgrid(*ranges)
new_points = np.random.rand(1000, n_features-1)
X_vals, y_vals = boundary_points[:,0:n_features-1], boundary_points[:,-1] 

interpolated_values = griddata(X_vals, y_vals, new_points, method='cubic')

4


InvalidIndexError: (slice(None, None, None), slice(0, 3, None))

In [9]:
from scipy.interpolate import RBFInterpolator 
print(boundary_points.shape)
n_features = X.shape[1]
X_vals, y_vals = boundary_points[:,0:n_features-1], np.reshape(boundary_points[:,-1], (-1,1)) 
print(y_vals.shape)
interpolator = RBFInterpolator(X_vals, y_vals, kernel='cubic', smoothing=1e-12)

(160000, 4)
(160000, 1)


MemoryError: (unable to allocate 18446744073541027856 bytes)

In [17]:
n_features = X.shape[1]
ranges = [np.linspace(boundary_points[:, j].min(), boundary_points[:, j].max(), 25) for j in range(n_features)]
grids = np.meshgrid(*ranges) 

grid_points = np.vstack([g.ravel() for g in grids]).T
print(grid_points.shape)

(390625, 4)


In [18]:
print(grid_points.shape)

(390625, 4)


In [19]:
eval_coords = grid_points[:,0:n_features-1]
print(eval_coords.shape)
eval_values = interpolator(eval_coords)
add_boundary_pts = np.hstack((eval_coords, eval_values))
print(eval_coords.shape)
print(eval_values.shape)
print(add_boundary_pts.shape)

(390625, 3)
(390625, 3)
(390625, 1)
(390625, 4)


# Interpolating Additional Boundary points Using N-1 data point coordinates 

In [20]:
arr = model.predict_proba(add_boundary_pts)

In [21]:
average_diff = np.mean(arr[:, 0] - arr[:, 1])
print("Average difference:", average_diff)

Average difference: -3.008925147444152e-06
